<a href="https://colab.research.google.com/github/shamim-hussain/wine-quality-classification-logistic-regression/blob/main/Wine_Quality_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>